Script for generating Figure 4:
- Fig. 4:(a) Circumpolar classification of cross-slope profiles into Fresh (green), Cool (blue), Warm (red), and Dense (purple) categories described in the main text, indicated with four colorbars corresponding to the \citet{Thompson2018} characterisation based on observations (`Obs'), and ACCESS-OM2-01 output from the control, MW45, and MW85 simulations. Model output is classified by inspecting transects of potential density and temperature crossing the 1000 m isobath, averaged over the final year of the experimental period. The control pattern does not drift over the 10 year experimental period. 74 transects, taken at intervals of 5$^\circ$ longitude, are used, with additional transects included along the western edge of the Weddell Sea where the 1000 m isobath bears north. Maps show upper 1000 m depth mean absolute salinity anomalies averaged over the final year of the experiment. Black contours are the 1000 m isobath. (b) Example transects from the control simulation illustrating the four shelf break regimes used in (a) are shown. Transect locations are indicated in (a) by corresponding colored bars.

categorisation is based on a large quantity of cross isobath transects which are archived. 
This is just the figure creation, not the analysis itself.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as col
import matplotlib.path as mpath
### establish link to python functions
import sys
sys.path.append('/home/156/rm2389/Honours_Thesis/python_functions/')
### import required python functions
from masking import shelf_mask_isobath
from spatial_averaging import month_weights
import cosima_cookbook as cc
from gsw import SA_from_SP, p_from_z, CT_from_pt, beta, sigma1
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from mpl_toolkits.axes_grid1.colorbar import colorbar
from matplotlib import rc, rcParams
rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
rc('text', usetex=True)
rc('xtick', labelsize=35) 
rc('ytick', labelsize=35) 
rc('axes', labelsize=35) 
from dask.distributed import Client
client = Client('tcp://localhost:8786')
import cmocean.cm as cmo
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
client

Client Scheduler: tcp://localhost:8786 Dashboard: http://localhost:8787/status,Cluster Workers: 6 Cores: 6 Memory: 18.00 GB


In [2]:
db = '/g/data/v45/rm2389/databases/freshwater_experiments.db' # database path
session = cc.database.create_session(db)

In [3]:
control='01deg_jra55v13_ryf8485_spinup6'
rcp45='01deg_jra55v13_ryf8485_freshRCP45'
rcp85='01deg_jra55v13_ryf8485_freshRCP85'

In [4]:
## data required for lat/lon mapping and bathymetry contours
ht = cc.querying.getvar(control,'ht',session, n=1) 
ht = ht.sel(yt_ocean = slice(-90,-55))
land_mask = np.squeeze(ht.values)
land_mask = land_mask * 0
land_mask[np.isnan(land_mask)] = 1
yt_ocean =ht.yt_ocean.values
xt_ocean =ht.xt_ocean.values
ht_shelf , shelf_mask = shelf_mask_isobath(ht)

Plot ASF categorisation in strips to be positioned above salinity map.

In [30]:
def ASC_strips(ax1, ax2, ax3, ax4):
    
    ## strip dimensions are determined by eye based off transects of annual average temperature at 
    ## the regions indicated by the bars in the map.
    
    plt.setp(ax1.get_xticklabels(), visible=False)
    plt.setp(ax1.get_yticklabels(), visible=False)
    plt.setp(ax2.get_xticklabels(), visible=False)
    plt.setp(ax2.get_yticklabels(), visible=False)
    plt.setp(ax3.get_xticklabels(), visible=False)
    plt.setp(ax3.get_yticklabels(), visible=False)
    plt.setp(ax4.get_xticklabels(), visible=False)
    plt.setp(ax4.get_yticklabels(), visible=False)
    plt.subplots_adjust(hspace=.1)

    ## THOMPSON et al 2018 COMPARISON (estimated from Figure 4)
    ## D regions
    ax1.fill_between([-29,-54],[0,0],[1,1], color = 'blueviolet')
    ax1.fill_between([-178,-228],[0,0],[1,1], color = 'blueviolet')
    ax1.fill_between([70,45],[0,0],[1,1], color = 'blueviolet')
    ## F regions
    ax1.fill_between([-54,-56],[0,0],[1,1], color = 'lightgreen')
    ax1.fill_between([-120,-178],[0,0],[1,1], color = 'lightgreen')
    ax1.fill_between([-228,-247],[0,0],[1,1], color = 'lightgreen')
    ax1.fill_between([-280,-258],[0,0],[1,1], color = 'lightgreen')
    ax1.fill_between([70,80],[0,0],[1,1], color = 'lightgreen')
    ax1.fill_between([-29,45],[0,0],[1,1], color = 'lightgreen')
    ## W regions
    ax1.fill_between([-120,-56],[0,0],[1,1], color = 'tomato')
    ax1.fill_between([-247,-258],[0,0],[1,1], color = 'tomato')
    ax1.axis('off')
    ax1.text(81,0.65, 'Obs', fontsize = 40)

    
    ## CONTROL PERIOD
    ax2.fill_between([52.5,80],[0,0],[1,1], color = 'blueviolet')
    ax2.fill_between([-32.5,52.5],[0,0],[1,1], color = 'lightgreen')
    ax2.fill_between([-62.5,-32.5],[0,0],[1,1], color = 'blueviolet')
    ax2.fill_between([-157.5,-62.5],[0,0],[1,1], color = 'tomato')
    ax2.fill_between([-177.5,-157.5],[0,0],[1,1], color = 'c')
    ax2.fill_between([-242.5,-177.5],[0,0],[1,1], color = 'blueviolet')
    ax2.fill_between([-277.5,-242.5],[0,0],[1,1], color = 'lightgreen')
    ax2.fill_between([-280,-277.5],[0,0],[1,1], color = 'blueviolet')
    ax2.axis('off')
    ax2.text(81,0.3, 'Control', fontsize = 40)
    
    ## RCP 4.5 YEAR 10
    ax3.fill_between([52.5,80],[0,0],[1,1], color = 'blueviolet')
    ax3.fill_between([-32.5,52.5],[0,0],[1,1], color = 'lightgreen')
    ax3.fill_between([-57.5,-32.5],[0,0],[1,1], color = 'blueviolet')
    ax3.fill_between([-67.5,-57.5],[0,0],[1,1], color = 'c')
    ax3.fill_between([-172.5,-67.5],[0,0],[1,1], color = 'tomato')
    ax3.fill_between([-187.5,-172.5],[0,0],[1,1], color = 'c')
    ax3.fill_between([-280,-187.5],[0,0],[1,1], color = 'lightgreen')
    ax3.axis('off')
    ax3.text(81,0.3, 'MW45', fontsize = 40)

    ## RCP8.5 YEAR 10 
    ax4.fill_between([-72.5, 80],[0,0],[1,1], color = 'lightgreen')
    ax4.fill_between([-142.5,-72.5],[0,0],[1,1], color = 'tomato')
    ax4.fill_between([-157.5,-142.5],[0,0],[1,1], color = 'lightgreen')
    ax4.fill_between([-172.5,-157.5],[0,0],[1,1], color = 'tomato')
    ax4.fill_between([-187.5,-172.5],[0,0],[1,1], color = 'c')
    ax4.fill_between([-280,-187.5],[0,0],[1,1], color = 'lightgreen')
    ax4.axis('off')
    ax4.axis('off')
    ax4.text(81,0.3, 'MW85', fontsize = 40)

Example transects

In [6]:
def warm_example(ax):
    ## example of transect I identify as "warm"
    filename = '/g/data/v45/rm2389/ASC_Transects/control/start_temp_transect_32.nc'
    temp_transect = xr.open_dataset(filename)
    temp_transect = temp_transect.pot_temp_transect
    norm1 = col.Normalize(vmin=-2.8,vmax=2.8)  
    plot =ax.pcolormesh(temp_transect.yt_ocean,temp_transect.st_ocean,temp_transect.values,cmap='RdBu_r',norm=norm1)
    levels = np.array([32.0, 32.2,32.4,32.42,32.44,32.46, 32.48,32.50])
    filename = '/g/data/v45/rm2389/ASC_Transects/control/start_sigma1_transect_32.nc'
    pot_rho_transect = xr.open_dataset(filename)
    pot_rho_transect = pot_rho_transect.pot_rho_1_transect
    ax.contour(pot_rho_transect.yt_ocean, pot_rho_transect.st_ocean, pot_rho_transect.values, levels=levels, colors = 'k', linewidths=1, alpha = 0.3)
    contours = ax.contour(pot_rho_transect.yt_ocean, pot_rho_transect.st_ocean, pot_rho_transect.values, levels=levels, colors = 'k', linewidths=1)
    ax.clabel(contours, inline = 1, fmt='%3.2f',fontsize = 22,manual = ((-73.6,50),(-72.5,100),(-72.5,300),(-72.5,400), (-72.5,500), (-72.5,700), (-72.5,800), (-72.5,1100)))
    plt.setp(ax.get_yticklabels(), visible=False)
    ax.set_xlabel('Latitude', fontsize = 35)
    ax.set_ylim(1500,0)
    return plot

In [25]:
def fresh_example(ax):
    ## example of transect I identify as "fresh"
    filename = '/g/data/v45/rm2389/ASC_Transects/control/start_temp_transect_58.nc'
    temp_transect = xr.open_dataset(filename)
    temp_transect = temp_transect.pot_temp_transect
    norm1 = col.Normalize(vmin=-2.8,vmax=2.8)  
    plot =ax.pcolormesh(temp_transect.yt_ocean,temp_transect.st_ocean,temp_transect.values,cmap='RdBu_r',norm=norm1)
    levels = np.array([32.4,32.45, 32.50, 32.52, 32.54,32.56])
    filename = '/g/data/v45/rm2389/ASC_Transects/control/start_sigma1_transect_58.nc'
    pot_rho_transect = xr.open_dataset(filename)
    pot_rho_transect = pot_rho_transect.pot_rho_1_transect
    ax.contour(pot_rho_transect.yt_ocean, pot_rho_transect.st_ocean, pot_rho_transect.values, levels=levels, colors = 'k', linewidths=1, alpha =0.3)
    contours = ax.contour(pot_rho_transect.yt_ocean, pot_rho_transect.st_ocean, pot_rho_transect.values, levels=levels, colors = 'k', linewidths=1)
#     ax.clabel(contours, inline = 1, fmt='%3.2f',fontsize = 22, manual = ((-64.3,50),(-65,200),(-64,300),(-64,500), (-64.5,700), (-64.5,1200)))
    ax.set_ylim(1500,0)
    ax.set_xlabel('Latitude', fontsize = 35)
    ax.set_ylabel('Depth (m)', fontsize = 35)
    return plot

In [24]:
def cool_example(ax):
    ## example of transect with my (new) categorisation of "cool"
    filename = '/g/data/v45/rm2389/ASC_Transects/control/start_temp_transect_40.nc'
    temp_transect = xr.open_dataset(filename)
    temp_transect = temp_transect.pot_temp_transect
    norm1 = col.Normalize(vmin=-2.8,vmax=2.8)  
    plot =ax.pcolormesh(temp_transect.yt_ocean,temp_transect.st_ocean,temp_transect.values,cmap='RdBu_r',norm=norm1)
    levels = np.array([32.3,32.38,32.42,32.44,32.46, 32.48,32.50])
    filename = '/g/data/v45/rm2389/ASC_Transects/control/start_sigma1_transect_40.nc'
    pot_rho_transect = xr.open_dataset(filename)
    pot_rho_transect = pot_rho_transect.pot_rho_1_transect
    ax.contour(pot_rho_transect.yt_ocean, pot_rho_transect.st_ocean, pot_rho_transect.values, levels=levels, colors = 'k', linewidths=1, alpha = 0.3)
    contours = ax.contour(pot_rho_transect.yt_ocean, pot_rho_transect.st_ocean, pot_rho_transect.values, levels=levels, colors = 'k', linewidths=1)
    ax.clabel(contours, inline = 1, fmt='%3.2f',fontsize = 22, manual = ((-76,100), (-75.5,200), (-75.5,400), (-75.5,500), (-75.5,600), (-75.5,800), (-75.5,1000)))
    plt.setp(ax.get_yticklabels(), visible=False)
    ax.set_ylim(1500,0)
    ax.set_xlabel('Latitude', fontsize = 35)
    return plot

In [9]:
def dense_example(ax):
    filename = '/g/data/v45/rm2389/ASC_Transects/control/start_temp_transect_1.nc'
    temp_transect = xr.open_dataset(filename)
    temp_transect = temp_transect.pot_temp_transect
    norm1 = col.Normalize(vmin=-2.8,vmax=2.8)  
    plot =ax.pcolormesh(temp_transect.yt_ocean,temp_transect.st_ocean,temp_transect.values,cmap='RdBu_r',norm=norm1)
    levels = np.array([32.4,32.50, 32.52, 32.54, 32.55])
    filename = '/g/data/v45/rm2389/ASC_Transects/control/start_sigma1_transect_1.nc'
    pot_rho_transect = xr.open_dataset(filename)
    pot_rho_transect = pot_rho_transect.pot_rho_1_transect
    ax.contour(pot_rho_transect.yt_ocean, pot_rho_transect.st_ocean, pot_rho_transect.values, levels=levels, colors = 'k', linewidths=1, alpha = 0.3)
    contours = ax.contour(pot_rho_transect.yt_ocean, pot_rho_transect.st_ocean, pot_rho_transect.values, levels=levels, colors = 'k', linewidths=1)
    ax.clabel(contours, inline = 1, fmt='%3.2f',fontsize = 22, manual = ((-66.5,50), (-66.0,100), (-66.5,300), (-66.5,700), (-66.5,1000)))
    plt.setp(ax.get_yticklabels(), visible=False)
    ax.set_ylim(1500,0)
    ax.set_xlabel('Latitude', fontsize = 35)
    return plot

Now to determine the upper 1000 m depth mean salinity anomaly

In [10]:
def depthmean_map(var, dzt, depth1, depth2):
    var = var.sel(st_ocean = slice(depth1,depth2))
    dzt = dzt.sel(st_ocean = slice(depth1,depth2))
    var_scaled = var * dzt;
    var_scaled_sum = var_scaled.sum(dim = 'st_ocean');
    sum_depth = dzt.sum(dim = 'st_ocean')
    var_weighted_mean = var_scaled_sum /sum_depth;
    return var_weighted_mean

In [11]:
timeslice = slice('1946-06','1947-05')
latslice = slice(-90,-59)
## final year and far southern

In [12]:
absolute_salt_control = xr.open_dataset('/g/data/v45/rm2389/Water_Mass_Tracers/AS_control_1946-06-1947-05.nc', chunks = {'time':1,'st_ocean':7, 'yt_ocean':300, 'xt_ocean':400})
absolute_salt_control = absolute_salt_control.salt
dzt_control = cc.querying.getvar(control,'dzt', session, n=-68)
dzt_control = dzt_control.sel(time = timeslice, yt_ocean = latslice)

In [13]:
absolute_salt_control_mean = depthmean_map(absolute_salt_control, dzt_control, 0, 1000)
absolute_salt_control_mean = absolute_salt_control_mean.load()

In [14]:
absolute_salt_rcp45 = xr.open_dataset('/g/data/v45/rm2389/Water_Mass_Tracers/AS_rcp45_1946-06-1947-05.nc', chunks = {'time':1,'st_ocean':None, 'yt_ocean':300, 'xt_ocean':400})
absolute_salt_rcp45 = absolute_salt_rcp45.salt.sel(yt_ocean = latslice)
dzt_rcp45 = cc.querying.getvar(rcp45,'dzt', session)
dzt_rcp45 = dzt_rcp45.sel(time = timeslice, yt_ocean = latslice).isel(time = slice(0,12))

In [15]:
absolute_salt_rcp45_mean = depthmean_map(absolute_salt_rcp45, dzt_rcp45, 0, 1000)
absolute_salt_rcp45_mean = absolute_salt_rcp45_mean.load()

In [16]:
absolute_salt_rcp85 = xr.open_dataset('/g/data/v45/rm2389/Water_Mass_Tracers/AS_rcp85_1946-06-1947-05.nc', chunks = {'time':1,'st_ocean':None, 'yt_ocean':300, 'xt_ocean':300})
absolute_salt_rcp85 = absolute_salt_rcp85.salt
absolute_salt_rcp85 = absolute_salt_rcp85.sel(yt_ocean = latslice)
dzt_rcp85 = cc.querying.getvar(rcp85,'dzt', session)
dzt_rcp85 = dzt_rcp85.sel(time = timeslice, yt_ocean = latslice)

In [17]:
absolute_salt_rcp85_mean = depthmean_map(absolute_salt_rcp85, dzt_rcp85, 0, 1000)
absolute_salt_rcp85_mean = absolute_salt_rcp85_mean.load()

In [18]:
## time mean
absolute_salt_control_mean_time = absolute_salt_control_mean.mean(dim = 'time').load()
absolute_salt_rcp45_mean_time = absolute_salt_rcp45_mean.mean(dim = 'time').load()
absolute_salt_rcp85_mean_time = absolute_salt_rcp85_mean.mean(dim = 'time').load()
## anomaly
absolute_salt_rcp45_anomaly = absolute_salt_rcp45_mean_time - absolute_salt_control_mean_time
absolute_salt_rcp85_anomaly = absolute_salt_rcp85_mean_time - absolute_salt_control_mean_time

In [19]:
## check range for plot 
print(absolute_salt_rcp45_anomaly.max().values)
print(absolute_salt_rcp45_anomaly.min().values)
print(absolute_salt_rcp85_anomaly.max().values)
print(absolute_salt_rcp85_anomaly.min().values)

0.21320870594952623
-6.005249355242899
0.0920447128047428
-13.070117303405425


In [22]:
def salinity_maps(ax0, ax1):
    transect_lons = [75,75,70,70,65,65,60,60,55,55,51,51,45,45,40,40,35,35,30,30,25,25,15,15,10,10,5,5,-5,-5,-10,-10,-15,-15,-22.5,-22.5,-30,-30,-35,-35,-40,-40,-45,-45,-50,-50,-85,-85,-90,-90,-95,-95,-100,-100,-105,-105,-110,-110,-115,-115,-120,-120,-125,-125,-130,-130,-135,-135,-140,-140,-145,-145,-150,-150,-155,-155,-160,-160,-165,-165, -170,-170,-175,-175,-180,-180,-185,-185,-190,-190,-195,-195,-200,-200,-205,-205,-210,-210,-215,-215,-220,-220,-225,-225,-230,-230,-235,-235,-240,-240,-245,-245,-250,-250,-255,-255,-260,-260,-265,-265,-270,-270,-275,-275,-279.8,-279.8,-54,-48,-58,-52,-58,-52]
    transect_lats = [-67.5,-65.5,-67.5,-65.5,-67.5,-65.5,-67.5,-65.5,-66,-64,-66.5,-64.55,-68,-66,-69,-67,-69,-67,-69.5,-67.5,-70.5,-68.5,-70,-68,-70.5,-68.5,-70.5,-68.5,-71,-69,-71.5,-69.5,-72.5,-70.5,-74.5,-72.5,-75,-73,-75,-73,-74.5,-72.5,-73.5,-71.5,-73,-71,-71,-69,-71.5,-69.5,-71.5,-69.5,-71.5,-69.5,-72,-70,-72,-70,-72.5,-70.5,-73,-71,-73,-71,-74,-72,-74.5,-72.5,-75,-73,-75.5,-73.5,-76.5,-74.5,-77,-75,-77,-75,-77,-75,-77,-75,-76,-74,-73.5,-71.5,-73,-71,-71.5,-69.5,-70.5,-68.5,-70,-68,-67.5,-65.5,-66.5,-64.5,-66.5,-64.5,-66.5,-64.5,-65.5,-63.5,-65.5,-63.5,-66,-64,-66,-64,-66,-64,-66,-64,-65.5,-63.5,-64.8,-62.8,-65.5,-63.5,-66.5,-64.5,-66.5,-64.5,-66.5,-64.5,-63.5,-63.5,-67,-67,-70,-70]

    norm1 = col.Normalize(vmin=-1.2,vmax=1.2) 
    salt = ax0.pcolormesh(absolute_salt_rcp45_anomaly.xt_ocean,absolute_salt_rcp45_anomaly.yt_ocean,absolute_salt_rcp45_anomaly.values,cmap='bwr',norm=norm1)
    ax0.contour(xt_ocean, yt_ocean,land_mask,[0,1], colors = 'k', linewidth = 1, alpha = 0.6)
    ax0.contour(ht_shelf.xt_ocean, ht_shelf.yt_ocean, shelf_mask.values, [0,1], colors = 'k', linewidth = 6)
    ax0.set_xlim((-280,80))
    ax0.set_ylim((-79,-59))
    ax0.set_ylabel('Latitude', fontsize = 45)
    plt.setp(ax0.get_xticklabels(), visible=False)
    salt = ax1.pcolormesh(absolute_salt_rcp85_anomaly.xt_ocean,absolute_salt_rcp85_anomaly.yt_ocean,absolute_salt_rcp85_anomaly.values,cmap='bwr',norm=norm1)
    ax1.contour(xt_ocean, yt_ocean,land_mask,[0,1], colors = 'k', linewidth = 1, alpha = 0.6)
    ax1.contour(ht_shelf.xt_ocean, ht_shelf.yt_ocean, shelf_mask.values, [0,1], colors = 'k', linewidth = 6)
    ax1.set_xlim((-280,80))
    ax1.set_ylim((-79,-59))
    ax1.set_ylabel('Latitude', fontsize = 45)
    ax1.set_xlabel('Longitude', fontsize = 45)
    i=1
    ax1.plot(transect_lons[2*i:2*i+2], transect_lats[2*i:2*i+2], color = 'blueviolet', linewidth = 8)
    ax0.plot(transect_lons[2*i:2*i+2], transect_lats[2*i:2*i+2], color = 'blueviolet', linewidth = 8)
    i=32
    ax1.plot(transect_lons[2*i:2*i+2], transect_lats[2*i:2*i+2], color = 'tomato', linewidth = 8)
    ax0.plot(transect_lons[2*i:2*i+2], transect_lats[2*i:2*i+2], color = 'tomato', linewidth = 8)
    i=40
    ax1.plot(transect_lons[2*i:2*i+2], transect_lats[2*i:2*i+2], color = 'c', linewidth = 8)
    ax0.plot(transect_lons[2*i:2*i+2], transect_lats[2*i:2*i+2], color = 'c', linewidth = 8)
    i=58
    ax1.plot(transect_lons[2*i:2*i+2], transect_lats[2*i:2*i+2], color = 'lightgreen', linewidth = 8)
    ax0.plot(transect_lons[2*i:2*i+2], transect_lats[2*i:2*i+2], color = 'lightgreen', linewidth = 8)
    return salt

In [ ]:
fig=plt.figure(1,(30,36))
gs1 = gridspec.GridSpec(6, 1, height_ratios=[1, 1, 1, 1, 10,10]) 
gs1.update(bottom = 0.42)

ax = plt.subplot(gs1[5])
ax0 = plt.subplot(gs1[4])
ax1 = plt.subplot(gs1[3], sharex = ax0)
ax2 = plt.subplot(gs1[2], sharex = ax0)
ax3 = plt.subplot(gs1[1], sharex = ax0)
ax4 = plt.subplot(gs1[0], sharex = ax0)

salt = salinity_maps(ax0, ax)
ASC_strips(ax1, ax2, ax3, ax4)

gs2 = gridspec.GridSpec(1, 4) 
gs2.update(top = 0.34, wspace = 0.1)

ax6 = plt.subplot(gs2[0])
ax7 = plt.subplot(gs2[1])
ax8 = plt.subplot(gs2[2])
ax9 = plt.subplot(gs2[3])

plot = fresh_example(ax6)
warm_example(ax7)
dense_example(ax8)
cool_example(ax9)

cax = plt.axes([0.92, 0.13, 0.02, 0.21])
cbar=fig.colorbar(plot, cax = cax ,orientation='vertical')
cbar.set_label(r'$\theta$ ($^\circ$C)', fontsize = 30)

cax = plt.axes([0.92, 0.49, 0.02, 0.23])
cbar=fig.colorbar(salt, cax = cax ,orientation='vertical', ticks = [-1.5,-1,-0.5,0,0.5, 1, 1.5])
cbar.set_label(r'$\Delta S_{\text{abs}}$ (g/kg)', fontsize = 45)

figurepath = 'Fig4.png'
fig.savefig(figurepath, dpi=None, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format='png',
            transparent=False, bbox_inches='tight', pad_inches=0.1,
            frameon=None)
plt.show()

Then I did some aestheric changes and added colored boxes for the identification of the shelf regimes offline in photoshop.